# Synacor Challenge VM

**Solution Started**: July 7, 2025  
**Author**: Abbas Moosajee

This notebook loads and runs the Synacor challenge binary using a custom `VirtualMachine` implementation, highlights challenge codes, and displays the interactive game output in styled HTML.

In [179]:
import os, re, time, copy, json
from collections import defaultdict, deque
from html import escape
from IPython.display import display, HTML
from VirtualMachine import VirtualMachine
start_time = time.time()

In [180]:
# Load the binary program file
program_file = "challenge.bin"
file_path = os.path.join(os.getcwd(), program_file)  # assumes file in current working dir
vm_program = open(file_path, "rb").read()

In [181]:
class SynacorConsole:
    track_backwards = {"north":"south", "south":"north", "east":"west","west":"east"}

    def __init__(self, software: bytes, spec_code: str, visualize: bool = True):
        self.software = software
        self.visualize = visualize
        self.console = self.reset_console()
        self.__extract_codes(" ", [spec_code])

    def reset_console(self):
        console = VirtualMachine(self.software)
        self.game_map = defaultdict(list)
        self.overall_commands = []
        self.challenge_codes = {}
        self.valid_codes = set()
        self.code_no = 0
        return console

    @staticmethod
    def md5_hash(code: str) -> str:
        import hashlib
        return hashlib.md5(code.encode('utf-8')).hexdigest()

    def create_room_id(self, text: str) -> int:
        h = self.md5_hash(text.strip().lower())
        # Take first 4 hex digits, convert to int, and mod 10000 to get a 4-digit number
        return int(h[:8], 16) % 10000

    def __extract_codes(self, text, codes = []):
        """Extract mixed-case challenge codes and store them with their MD5 hashes."""
        codes_found = re.findall(r"[A-Za-z]*[a-z]+[A-Z]+[a-z]+[A-Za-z]*", text)
        codes.extend(codes_found)

        new_codes = []
        for code in codes:
            if code not in self.valid_codes:
                self.code_no += 1
                self.challenge_codes[self.code_no] = (code, self.md5_hash(code))
                new_codes.append(code)
                self.valid_codes.add(code)
        return new_codes

    def __format_terminal(self, text, code_color, input_color):
        """Highlight any detected code-like strings in the terminal."""
        lines = []
        for line in text.splitlines():
            # Simple input highlight: lines starting with '>'
            if line.strip().startswith('>>'):
                line = f"<span style='color:{input_color}'>{line}</span>"
            else:
                line = re.sub(      # Example: highlight code-like patterns
                    r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
                    rf"<span style='color:{code_color}'>\1</span>", line
                )
            lines.append(line)
        return "\n".join(lines)

    def display_terminal(self, text, actions):
        """Format and display terminal output with highlighted codes and interleaved actions."""
        self.color_dict = {
            "background": "#282828", "terminal": "#33FF00",
            "codes": "#FF0000", "input": "#0073FF"
        }

        # Normalize line endings and remove excessive empty lines
        text = re.sub(r'\n{3,}', '\n\n', text.strip())  # At most 2 newlines
        parts = re.split(r'(What do you do\?)', text)

        output_lines = []

        if actions:
            output_lines.append(f">> {actions[0]}")
        action_index = 1 if actions else 0

        # Now interleave remaining actions after each prompt
        i = 0
        while i < len(parts):
            output_lines.append(parts[i])
            input_prompt = "What do you do?"
            if i + 1 < len(parts) and parts[i + 1] == input_prompt:
                output_lines.append(input_prompt) # Append prompt
                if action_index < len(actions):   # Append next action if available
                    output_lines.append(f">> {actions[action_index]}")
                    action_index += 1
                i += 2
            else:
                i += 1

        combined_text = "\n".join(output_lines)
        longest_line = max(max(len(line) for line in combined_text.split('\n')), 150)

        html = (
            f"<div style='background-color: {self.color_dict['background']}; "
            f"width: {longest_line + 2}ch; padding: 0.75ex;'>"
            f"<pre style='background-color: {self.color_dict['background']}; "
            f"color: {self.color_dict['terminal']}; margin: 0; font-family: monospace; "
            f"width: {longest_line + 2}ch;'>"
            + self.__format_terminal(combined_text, self.color_dict['codes'], self.color_dict['input'])
            + "</pre></div>"
        )

        display(HTML(html))

    def play_game_manually(self, actions = []):
        """
        Game Instructions:
        - look: You may merely 'look' to examine the room, or you may 'look ' (such as 'look chair') to examine something specific.
        - go: You may 'go ' to travel in that direction (such as 'go west'), or you may merely '' (such as 'west').
        - inv: To see the contents of your inventory, merely 'inv'.
        - take: You may 'take ' (such as 'take large rock').
        - drop: To drop something in your inventory, you may 'drop '.
        - use: You may activate or otherwise apply an item with 'use '.
        """
        if actions:
            self.overall_commands.extend(actions)
        full_terminal = self.console.run_computer(actions)
        current_terminal = full_terminal[-1]
        self.display_terminal(current_terminal, actions)
        self.__extract_codes(current_terminal)
        return self.overall_commands

    def __parse_game_state(self, terminal):
        lines = terminal.splitlines()
        location, description, things, exits = ("", "", [], [])

        current_section = None
        for line_no, line in enumerate(lines):
            line = line.strip()
            if line.startswith("==") and line.endswith("=="):
                location = line.strip("= ").strip()
                description = lines[line_no + 1]
            elif line.startswith("Things of interest here:"):
                current_section = "things"
            elif line.startswith("There") and "exit" in line:
                current_section = "exits"
            elif line.startswith("-") and current_section:
                item = line[2:].strip()
                if current_section == "things":
                    things.append(item)
                elif current_section == "exits":
                    exits.append(item)
            else:
                current_section = None

        return location, description, things, exits

    def __generate_commands(self, current_state, visited, commands_list):
        location, desc, all_items, exits = self.__parse_game_state(current_state)
        print(self.md5_hash(desc))
        if all_items:
            for item in all_items:
                commands_list.append(f"take {item}")
                commands_list.append(f"use {item}")
        commands_list.append(f"{exits[0]}")
        self.game_map[location].extend(exits)
        # print(f"{self.past_loc}, {location}")
        self.past_loc = location
        # print(location, exits)
        visited.add((location, tuple(exits)))
        # commands_list.append("help")
        return commands_list, visited

    def nested_dict(self):
        return defaultdict(self.nested_dict)

    def add_path(self, map_dict, path):
        current = map_dict
        for step in path:
            current = current[step]

    def __use_items(self, item, exits):
        """Return actions to collect and use the item, and what items will be added to inventory."""
        if item == 'tablet':
            return [f"take {item}", f"use {item}"], {item}
        elif item == 'empty lantern':
            return [f"take {item}", exits[0], exits[0]], {item}
        elif item == 'can':
            return [f"take {item}", f"use {item}", "use lantern"], {item, "lantern"}
        else:
            return [f"take {item}", f"use {item}"], {item}

    def bfs(self):
        """Perform BFS to explore and map out the game world."""
        BASE_ROOM = "Foothills"
        MAX_STEPS = 250

        steps = 0
        game_map = self.nested_dict()
        visited = set()
        room_info = {}
        inventory = set()
        item_paths = {}

        # Each queue entry: (console_state, pending_actions, path_so_far, full_action_history)
        queue = [(self.console.replicate(0), [], [], [])]

        while queue and steps < MAX_STEPS:
            steps += 1
            state, pending_actions, path, history = queue.pop(0)
            last_room = path[-1] if path else ""
            reverse_dir = self.track_backwards.get(last_room, "")

            # Run actions on replicated game state
            game_copy = state.replicate(steps)
            output = game_copy.run_computer(pending_actions)
            terminal_output = output[-1]
            self.__extract_codes(terminal_output)

            if len(self.challenge_codes) >= 5:
                print(history)
                # Uncomment for visual debug:
                # if self.visualize:
                #     self.display_terminal('\n\n'.join(output[1:]), history)
                # break

            # Parse the game state
            room, desc, items, exits = self.__parse_game_state(terminal_output)
            room_id = self.create_room_id(desc)
            room_info[room_id] = (room, desc)

            if (room_id, tuple(inventory)) in visited:
                continue
            visited.add((room_id, tuple(inventory)))

            # If there’s a new item to collect
            if items:
                item = items[0]
                if item not in inventory:
                    print(f"[{room}] Found item: {item}, Exits: {exits}, Inventory: {inventory}")
                    if item not in item_paths:
                        item_paths[item] = history

                    item_actions, collected = self.__use_items(item, exits)
                    inventory.update(collected)
                    combined_actions = history + item_actions
                    queue.append((game_copy, item_actions, path, combined_actions))
                    # continue  # Reprocess with updated state after taking item

            # Explore new directions
            for direction in exits:
                if direction == reverse_dir and items:
                    continue  # Avoid immediately going back after item pickup

                next_path = path + [direction]
                next_pending = [direction]
                next_history = history + [direction]
                self.add_path(game_map, next_path)
                queue.append((game_copy, next_pending, next_path, next_history))
        print(steps)

    def bfs(self):
        """Perform BFS to explore and map out the game world."""
        BASE_ROOM = "Foothills"
        MAX_STEPS = 250

        steps = 0
        game_map = self.nested_dict()
        visited = set()
        room_info = {}
        inventory = set()
        item_paths = {}

        # Each queue entry: (console_state, pending_actions, path_so_far, full_action_history)
        queue = [(self.console.replicate(0), [], [], [])]

        while queue and steps < MAX_STEPS:
            steps += 1
            state, pending_actions, path, actions_history = queue.pop(0)
            last_room = path[-1] if path else ""
            print(actions_history)
            reverse_dir = self.track_backwards.get(last_room, "")
            actions = []

            # Run actions on replicated game state
            game_copy = state.replicate(steps)
            output = game_copy.run_computer(pending_actions)
            terminal_output = output[-1]
            self.__extract_codes(terminal_output)

            # if len(self.challenge_codes) >= 5:
            #     print(actions_history)
            #     # Uncomment for visual debug:
            #     if self.visualize:
            #         self.display_terminal('\n\n'.join(output[1:]), actions_history)
            #     break

            # Parse the game state
            room, desc, items, room_exits = self.__parse_game_state(terminal_output)
            room_id = self.create_room_id(desc)
            room_info[room_id] = (room, desc)

            if (room_id, tuple(inventory)) in visited:
                continue
            visited.add((room_id, tuple(inventory)))
            # # If there's an item, pick it up
            if items:
                item = items[0]
                print(room, item, room_exits, inventory)
                if item not in item_paths:
                    item_paths[item] = actions_history
                item_actions, collected = self.__use_items(item, room_exits)
                inventory.update(collected)
                actions.extend(item_actions)
                actions_history.extend(item_actions)
                queue.append((game_copy, item_actions, path, actions_history))

            # Explore new directions
            for direction in room_exits:
                if direction == reverse_dir and items:
                    continue  # Don't go backwards immediately

                next_path = path + [direction]
                next_actions = actions + [direction]
                all_actions = actions_history + [direction]
                self.add_path(game_map, next_path)
                queue.append((game_copy, next_actions, next_path, all_actions))

        # Optional debug output
        print("steps:", steps)
        # self.print_game_map(game_map)
        # print(item_paths)
        return

    def bfs1(self):
        """Perform BFS to explore and map out the game world."""
        steps = 0
        BASE_ROOM = "Foothills"
        game_map = self.nested_dict()
        visited = set()
        room_info = {}
        inventory = set()
        item_paths = {}

        # Each queue entry: (console_state, actions_taken, path_taken, action_history)
        queue = [(self.console.replicate(0), [], [], [])]

        while queue and steps < 250:  # Limit steps for testing/debugging
            steps += 1
            state, input_actions, path, action_history = queue.pop(0)
            # print(action_history)
            last_room = path[-1] if path else ""
            reverse_dir = self.track_backwards.get(last_room, "")

            # Prepare a clean game state and run the latest action
            game_copy = state.replicate(steps)
            output = game_copy.run_computer(input_actions)
            terminal_output = output[-1]
            self.__extract_codes(terminal_output)
            actions = []
            if len(self.challenge_codes) >= 5:
                print(action_history)
            #     if self.visualize:
            #         full_terminal = '\n\n'.join(output[1:])
            #         self.display_terminal(full_terminal, action_history)
            #     break
            # Parse current state
            room, desc, items, room_exits = self.__parse_game_state(terminal_output)

            if room == "Ruins":
                print(room, desc)
            room_id = self.create_room_id(desc)

            if (room_id, tuple(inventory)) in visited:
                continue
            visited.add((room_id, tuple(inventory)))
            room_info[room_id] = (room, desc)

            if items and items[0] in inventory:
                continue
            # # If there's an item, pick it up
            if items:
                item = items[0]
                print(room, item, room_exits, inventory)
                if item not in item_paths:
                    item_paths[item] = action_history
                item_actions, collected = self.__use_items(item, room_exits)
                inventory.update(collected)
                actions.extend(item_actions)
                action_history.extend(item_actions)
                # queue.append((game_copy, actions, path, action_history))

            # Explore new directions
            for direction in room_exits:
                if direction == reverse_dir and items:
                    continue  # Don't go backwards immediately

                next_path = path + [direction]
                next_actions = actions + [direction]
                all_actions = action_history + [direction]
                self.add_path(game_map, next_path)
                queue.append((game_copy, next_actions, next_path, all_actions))

        # Optional debug output
        print("steps:", steps)
        # self.print_game_map(game_map)
        # print(item_paths)
        return

    def bfs1(self):
        """Perform BFS to explore and map out the game world."""
        steps = 0
        BASE_ROOM = "Foothills"
        game_map = self.nested_dict()
        visited = set()
        room_info = {}
        inventory = set()
        item_paths = {}

        # Each queue entry: (console_state, actions_taken, path_taken, action_history)
        queue = [(self.console.replicate(0), [], [], [])]

        while queue and steps < 150:  # Limit steps for testing/debugging
            steps += 1
            state, input_actions, path, action_history = queue.pop(0)
            # print(action_history)
            last_room = path[-1] if path else ""
            reverse_dir = self.track_backwards.get(last_room, "")

            # Prepare a clean game state and run the latest action
            game_copy = state.replicate(steps)
            output = game_copy.run_computer(input_actions)
            terminal_output = output[-1]
            self.__extract_codes(terminal_output)
            actions = []
            if len(self.challenge_codes) >= 5:
                print(action_history)
            #     if self.visualize:
            #         full_terminal = '\n\n'.join(output[1:])
            #         self.display_terminal(full_terminal, action_history)
            #     break
            # Parse current state
            room, desc, items, room_exits = self.__parse_game_state(terminal_output)

            if room == "Ruins":
                print(room, desc)
            room_id = self.create_room_id(desc)

            if (room_id, tuple(inventory)) in visited:
                continue
            visited.add((room_id, tuple(inventory)))
            room_info[room_id] = (room, desc)

            if items and items[0] in inventory:
                continue
            # # If there's an item, pick it up
            if items:
                item = items[0]
                print(room, item, room_exits, inventory)
                if item not in item_paths:
                    item_paths[item] = action_history
                item_actions, collected = self.__use_items(item, room_exits)
                inventory.update(collected)
                actions.extend(item_actions)
                action_history.extend(item_actions)
                # queue.append((game_copy, actions, path, action_history))

            # Explore new directions
            for direction in room_exits:
                if direction == reverse_dir and items:
                    continue  # Don't go backwards immediately

                next_path = path + [direction]
                next_actions = actions + [direction]
                all_actions = action_history + [direction]
                self.add_path(game_map, next_path)
                queue.append((game_copy, next_actions, next_path, all_actions))

    def print_game_map(self, d=None, indent=2):
        """Pretty print the nested defaultdict tree as a standard dict."""
        if d is None:
            d = self.game_map  # or whatever your map variable is
        def to_dict(d):
            if isinstance(d, defaultdict):
                return {k: to_dict(v) for k, v in d.items()}
            return d
        print(json.dumps(to_dict(d), indent=indent))

    def play_game(self, actions = []):
        steps = 0
        self.past_loc = "Base"
        game_state = self.console.replicate()
        visited = set()

        while True:
            if actions:
                self.overall_commands.extend(actions)
            full_terminal = game_state.run_computer(actions)
            current_terminal = full_terminal[-1]
            self.__extract_codes(current_terminal)
            if self.visualize:
                self.display_terminal(current_terminal, actions)
            if steps >= 7:
                break
            actions, visited = self.__generate_commands(current_terminal, visited, [])
            steps += 1
        print(self.overall_commands)

        return self.overall_commands


In [182]:
console = SynacorConsole(vm_program, "LDOb7UGhTi")
# command_list = console.play_game()
test = console.bfs()

[]
Foothills tablet ['doorway', 'south'] set()
['take tablet', 'use tablet']
['take tablet', 'use tablet', 'doorway']
['take tablet', 'use tablet', 'south']
['take tablet', 'use tablet', 'doorway', 'north']
['take tablet', 'use tablet', 'doorway', 'south']
['take tablet', 'use tablet', 'south', 'north']
['take tablet', 'use tablet', 'doorway', 'north', 'north']
['take tablet', 'use tablet', 'doorway', 'north', 'south']
['take tablet', 'use tablet', 'doorway', 'south', 'doorway']
['take tablet', 'use tablet', 'doorway', 'south', 'south']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'south']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'back']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down']
['take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'con

In [183]:
# manual_console = SynacorConsole(vm_program, "LDOb7UGhTi")
# manual_console.play_game_manually()
# game_actions = [
#     # 'take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west',
#     # 'passage', 'ladder', 'west', 'south', 'north', 'take can', 'use can', 'use lantern',
#     # 'west', 'west', 'south', 'north', 'west'
#     'take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west',
#     'passage', 'ladder', 'west', 'south', 'north', 'take can', 'use can', 'use lantern',
#     'west', 'west', 'south', 'north', 'west', 'east', 'east', 'forward', 'run', 'run'
#     ]
# all_actions = manual_console.play_game_manually(game_actions)


In [184]:
for code_no, code_group in console.challenge_codes.items():
    print(code_no, code_group)
print(f"Execution Time = {time.time() - start_time:.5f}s")

1 ('LDOb7UGhTi', '76ec2408e8fe3f1753c25db51efd8eb3')
2 ('ImoFztWQCvxj', '0e6aa7be1f68d930926d72b3741a145c')
3 ('BNCyODLfQkIl', '7997a3b2941eab92c1c0345d5747b420')
4 ('pWDWTEfURAdS', '186f842951c0dcfe8838af1e7222b7d4')
5 ('rdMkyZhveeIv', '2bf84e54b95ce97aefd9fc920451fc45')
Execution Time = 1.91133s
